# Imports

In [1]:
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import math
import numpy as np

# Download dataset

In [2]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data

--2021-03-27 12:19:31--  https://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 712565 (696K) [application/x-httpd-php]
Saving to: ‘letter-recognition.data.4’

letter-recognition. 100%[===================>] 695.86K  2.59MB/s    in 0.3s    

2021-03-27 12:19:32 (2.59 MB/s) - ‘letter-recognition.data.4’ saved [712565/712565]



In [3]:
df = pd.read_csv("/content/letter-recognition.data", header = None)
df.columns = ['label', 'xbox', 'ybox', 'width', 'height', 'onpix', 'xbar','ybar', 'x2bar', 'y2bar', 'xybar', 'x2ybar', 'xy2bar', 'xedge','xedgey', 'yedge', 'yedgex']
df

,label,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,D,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,C,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,T,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,S,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


# Decision Tree Implementation

In [4]:
class Node:
    def __init__(self, L, R, a_prime):
        self.L = L
        self.R = R
        self.a_prime = a_prime
        self.children = []

In [5]:
class Tree:
    def __init__(self, X, y, gamma, alpha, beta, nu):
        self.gamma = gamma
        self.alpha = alpha
        self.beta = beta
        self.nu = nu
        self.n_samples, self.n_features = X.shape
        self.A = X.columns
        eligible = dict((a, True) for a in self.A)
        X = X
        self.root = self.build_tree(float('-inf'), float('inf'), X, 0, eligible)
    
    def build_tree(self, left, right, X, prev_A_tr, eligible):
        
        # 2.5 Stopping Conditions
        n_t = len(X)

        if n_t == 0:
            return None

        min_proxy_impurity_decrease = float('inf')
        a_t = None
        children = None
        Y_t = None

        for a in self.A:
            # 2.4 Pre-pruning mechanism
            if eligible[a] == False:
                continue

            if len(np.unique(X[a])) == 1:
                continue
            
            # 2.1 Density Estimation
            f, x = self.KDE(X, a)

            # 2.2 Division
            # (a) Clipping KDE
            threshold = self.gamma * max(f)

            Y = []
            i = 0
            while i < len(f):
                while i < len(f) and f[i] < threshold:
                    i += 1

                if i == len(f):
                    break

                l = i
                i += 1

                while i < len(f) and f[i] >= threshold:
                    i += 1
                
                r = i - 1
                Y.append([l, r])
                i += 1

            # (b) Revision
            Y_prime = []
            for [l, r] in Y:
                i = l + 1
                L, R = l, r
                while i < r:
                    while i < r and f[i] < f[i + 1]:
                        i += 1
                    
                    if i == r:
                        continue
                    
                    maxima1 = i
                    i += 1

                    while i < r and f[i] > f[i + 1]:
                        i += 1
                    
                    if i == r:
                        continue

                    minima = i
                    i += 1

                    while i < r and f[i] < f[i + 1]:
                        i += 1

                    if i == r:
                        continue

                    maxima2 = i
                    
                    if f[minima] <= alpha * min(f[maxima1], f[maxima2]):
                        Y_prime.append([L, i])
                        L = i
                Y_prime.append([L, R])
            

            # (c) Assessment
            Y_prime[:] = [[l, r] for [l,r] in Y_prime if len(X.loc[(x[l] <= X[a]) & (X[a] <= x[r])]) > self.beta * self.n_samples]

            Y_prime_prime = []
            for [l, r] in Y_prime:
                sub_interval = X.loc[(x[l] <= X[a]) & (X[a] <= x[r])]
                Y_prime_prime.append([sub_interval[a].min(), sub_interval[a].max()])

            # 2.3 Impurity Decrease
            proxy_impurity_decrease = 0
            for [l_val, r_val] in Y_prime_prime:
                # print(l_val, r_val)
                n_t_i = len(X.loc[(l_val <= X[a]) & (X[a] <= r_val)])
                n_t_i_prime = n_t_i * (r_val - l_val) / (X[a].max() - X[a].min())
                proxy_impurity_decrease += (n_t_i * n_t_i_prime) / (n_t_i + n_t_i_prime)


            if proxy_impurity_decrease < min_proxy_impurity_decrease:
                min_proxy_impurity_decrease = proxy_impurity_decrease
                a_t = a
                Y_t = Y_prime_prime

        if a_t and len(Y_t) > 0:
            A_tr = len(X.loc[(Y_t[0][0] <= X[a]) & (X[a] <= Y_t[-1][1])]) / len(X)
            if A_tr == prev_A_tr:
                return None
            root = Node(left, right, a_t)
            eligible[a] = False
            for [l, r] in Y_t:
                child = self.build_tree(l, r, X.loc[(l <= X[a]) & (X[a] <= r)], A_tr, eligible)
                if child:
                    root.children.append(child)
        else:
            root = None

        return root

        
    def KDE(self, X, a):
        n_t = len(X)
        Q1 = X[a].quantile(0.25)
        Q3 = X[a].quantile(0.75)
        IQR = Q3 - Q1

        if IQR != 0:
            h_t = 0.9 * min(X[a].std(), IQR / 1.34) * (n_t ** (-1 / 5))
        else:
            h_t = 0.9 * X[a].std() * (n_t ** (-1 / 5))

        lo, hi = X[a].min(), X[a].max()
        x, f = [], []
        density_points = 512
        increment = (hi - lo) / density_points

        i = lo
        while i <= hi:
            x.append(i)
            i += increment

        for z in x:
            f.append(0)
            for x_i in X[a]:
                f[-1] += self.K((z - x_i) / h_t)
            f[-1] = (1 / (n_t * h_t)) * f[-1]

        return f, x
    

    def K(self, y):
        return 1 / math.sqrt(2 * math.pi) * math.exp(-(y ** 2) / 2)
    

    def predict(self, X):
        predictions = []
        for i, x in X.iterrows():
            copy = self.root
            predictions.append(self.make_prediction(x, copy))

        return predictions
    
    def make_prediction(self, x, root):
        if len(root.children) == 0:
            return 1

        for child in root.children:
            if root.L <= x[root.a_prime] <= root.R:
                return self.make_prediction(x, child)

        return 0

In [6]:
class OCC:
    def __init__(self, gamma, alpha, beta, nu):
        self.gamma = gamma
        self.alpha = alpha
        self.beta = beta
        self.nu = nu

    def fit(self, X, y):
        self.tree_ = Tree(X, y, gamma, alpha, beta, nu)

    def predict(self, X):
        return self.tree_.predict(X)

# Training and Testing

In [7]:
target_class = 'A'
df['label'] = (df['label'] == target_class).astype(int)
df

,label,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,0,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,0,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,0,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,0,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,0,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,0,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,0,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,0,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


In [8]:
target_class_data = df.loc[df['label'] == 1]
target_class_data

,label,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
7,1,1,1,3,2,1,8,2,2,2,8,2,8,1,6,2,7
78,1,3,7,5,5,3,12,2,3,2,10,2,9,2,6,3,8
118,1,3,8,5,6,3,9,2,2,3,8,2,8,2,6,3,7
130,1,2,1,4,2,1,8,1,2,2,7,2,8,2,5,2,7
134,1,3,7,5,5,3,10,4,1,2,8,3,9,2,4,2,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19893,1,3,7,4,5,2,7,4,2,0,6,2,8,2,7,1,7
19951,1,6,10,7,6,3,12,0,4,1,11,4,12,4,4,3,11
19966,1,2,3,3,1,1,6,2,2,1,5,2,8,1,6,1,7
19977,1,3,9,5,6,2,6,5,3,1,6,1,8,2,7,2,7


In [9]:
outlier_data = df.loc[df['label'] == 0]
outlier_data

,label,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,0,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,0,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,0,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,0,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,0,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19994,0,5,8,7,7,7,7,9,4,8,7,7,8,3,10,8,6
19995,0,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,0,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,0,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4


Noise Level = 2%

In [10]:
outlier_percentage = 2
m = len(target_class_data)
num_outliers = m * outlier_percentage // 100
included_outliers = outlier_data.sample(n = num_outliers)

In [11]:
data = target_class_data.append(included_outliers)
data

,label,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
7,1,1,1,3,2,1,8,2,2,2,8,2,8,1,6,2,7
78,1,3,7,5,5,3,12,2,3,2,10,2,9,2,6,3,8
118,1,3,8,5,6,3,9,2,2,3,8,2,8,2,6,3,7
130,1,2,1,4,2,1,8,1,2,2,7,2,8,2,5,2,7
134,1,3,7,5,5,3,10,4,1,2,8,3,9,2,4,2,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13521,0,2,7,3,5,1,7,8,0,7,13,6,7,0,8,1,7
15960,0,4,9,4,6,2,4,12,2,8,12,10,5,0,10,2,5
12637,0,4,6,6,4,3,6,9,2,8,11,8,6,1,8,6,5
18887,0,6,11,6,6,4,10,7,2,5,10,5,6,4,8,6,9


In [12]:
X = data.drop('label', 1)
y = data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1000)

In [13]:
gamma = 0.05
alpha = 0.5
beta = 0.02
nu = 0.05

clf = OCC(gamma, alpha, beta, nu)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("Classification report - \n", classification_report(y_test, y_pred))


Classification report - 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.98      1.00      0.99       262

    accuracy                           0.98       266
   macro avg       0.49      0.50      0.50       266
weighted avg       0.97      0.98      0.98       266



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Noise Level = 5%

In [14]:
outlier_percentage = 5
m = len(target_class_data)
num_outliers = m * outlier_percentage // 100
included_outliers = outlier_data.sample(n = num_outliers)

In [15]:
data = target_class_data.append(included_outliers)
data

,label,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
7,1,1,1,3,2,1,8,2,2,2,8,2,8,1,6,2,7
78,1,3,7,5,5,3,12,2,3,2,10,2,9,2,6,3,8
118,1,3,8,5,6,3,9,2,2,3,8,2,8,2,6,3,7
130,1,2,1,4,2,1,8,1,2,2,7,2,8,2,5,2,7
134,1,3,7,5,5,3,10,4,1,2,8,3,9,2,4,2,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,0,1,3,2,2,1,10,7,1,5,11,4,8,0,7,0,7
2381,0,2,3,3,2,2,6,7,4,7,6,6,10,3,8,4,9
7179,0,6,11,9,8,6,2,9,3,7,11,12,12,3,8,4,5
17144,0,5,7,6,9,6,8,5,8,4,6,5,9,4,8,6,9


In [16]:
X = data.drop('label', 1)
y = data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1000)

In [17]:
gamma = 0.05
alpha = 0.5
beta = 0.02
nu = 0.05

clf = OCC(gamma, alpha, beta, nu)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("Classification report - \n", classification_report(y_test, y_pred))

Classification report - 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        12
           1       0.96      1.00      0.98       262

    accuracy                           0.96       274
   macro avg       0.48      0.50      0.49       274
weighted avg       0.91      0.96      0.93       274



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Noise Level = 10%

In [18]:
outlier_percentage = 10
m = len(target_class_data)
num_outliers = m * outlier_percentage // 100
included_outliers = outlier_data.sample(n = num_outliers)

In [19]:
data = target_class_data.append(included_outliers)
data

,label,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
7,1,1,1,3,2,1,8,2,2,2,8,2,8,1,6,2,7
78,1,3,7,5,5,3,12,2,3,2,10,2,9,2,6,3,8
118,1,3,8,5,6,3,9,2,2,3,8,2,8,2,6,3,7
130,1,2,1,4,2,1,8,1,2,2,7,2,8,2,5,2,7
134,1,3,7,5,5,3,10,4,1,2,8,3,9,2,4,2,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18511,0,7,13,6,7,3,10,3,3,5,10,1,9,3,6,5,10
12768,0,6,11,8,8,7,7,7,7,8,7,6,5,6,8,4,7
14744,0,2,11,3,8,2,11,6,1,8,11,2,6,0,7,1,7
11286,0,3,2,4,3,2,7,8,5,4,7,6,7,5,9,2,6


In [20]:
X = data.drop('label', 1)
y = data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1000)

In [21]:
gamma = 0.05
alpha = 0.5
beta = 0.02
nu = 0.05

clf = OCC(gamma, alpha, beta, nu)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("Classification report - \n", classification_report(y_test, y_pred))

Classification report - 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        25
           1       0.91      1.00      0.95       262

    accuracy                           0.91       287
   macro avg       0.46      0.50      0.48       287
weighted avg       0.83      0.91      0.87       287



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Noise Level = 15%

In [22]:
outlier_percentage = 15
m = len(target_class_data)
num_outliers = m * outlier_percentage // 100
included_outliers = outlier_data.sample(n = num_outliers)

In [23]:
data = target_class_data.append(included_outliers)
data

,label,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
7,1,1,1,3,2,1,8,2,2,2,8,2,8,1,6,2,7
78,1,3,7,5,5,3,12,2,3,2,10,2,9,2,6,3,8
118,1,3,8,5,6,3,9,2,2,3,8,2,8,2,6,3,7
130,1,2,1,4,2,1,8,1,2,2,7,2,8,2,5,2,7
134,1,3,7,5,5,3,10,4,1,2,8,3,9,2,4,2,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5266,0,4,4,5,3,4,6,7,6,6,7,6,10,3,8,3,9
9751,0,4,10,6,7,3,4,13,4,4,13,8,3,1,10,2,5
17046,0,6,12,6,6,4,6,10,3,4,12,6,4,3,11,5,6
7298,0,5,9,7,6,5,9,8,2,5,13,5,5,2,9,3,9


In [24]:
X = data.drop('label', 1)
y = data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1000)

In [25]:
gamma = 0.05
alpha = 0.5
beta = 0.02
nu = 0.05

clf = OCC(gamma, alpha, beta, nu)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("Classification report - \n", classification_report(y_test, y_pred))

Classification report - 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        35
           1       0.88      1.00      0.94       265

    accuracy                           0.88       300
   macro avg       0.44      0.50      0.47       300
weighted avg       0.78      0.88      0.83       300



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
